In [31]:
#Anushri Ahir , 22BDS0394

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

titles = [book.h3.a['title'] for book in soup.select('article.product_pod')]
prices = [book.select_one('.price_color').text for book in soup.select('article.product_pod')]
availability = [book.select_one('.availability').text.strip() for book in soup.select('article.product_pod')]

books_df = pd.DataFrame({
    'title': titles,
    'price': prices,
    'availability': availability
})
books_df.head()


,title,price,availability
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock


In [2]:
import json

api_url = "https://openlibrary.org/search.json?q=harry+potter"
response = requests.get(api_url)
data = response.json()

docs = data['docs'][:10]
api_df = pd.DataFrame([{
    'title': doc.get('title'),
    'author': doc.get('author_name', ['N/A'])[0],
    'first_publish_year': doc.get('first_publish_year', 'N/A')
} for doc in docs])

api_df.head()


,title,author,first_publish_year
0,Harry Potter and the Philosopher's Stone,J. K. Rowling,1997
1,Harry Potter and the Order of the Phoenix,J. K. Rowling,2003
2,Harry Potter and the Deathly Hallows,J. K. Rowling,2007
3,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999
4,Harry Potter and the Chamber of Secrets,J. K. Rowling,1998


In [18]:
import pandas as pd

# Load CSVs
df1 = pd.read_csv("source1.csv")
df2 = pd.read_csv("source2.csv")

# Display missing values
print("👉 Missing values in df1:\n", df1.isnull().sum())
print("\n👉 Missing values in df2:\n", df2.isnull().sum())

# Fill missing price in df1 with mean
if df1['price'].isnull().any():
    df1['price'] = df1['price'].fillna(df1['price'].mean())
    print("\n✅ Filled missing 'price' in df1 with mean")

# Drop rows in df1 where 'title' is missing
if df1['title'].isnull().any():
    df1 = df1.dropna(subset=['title'])
    print("✅ Dropped rows with missing 'title' in df1")

# Fill missing availability with "Unknown" in both DataFrames
if df1['availability'].isnull().any():
    df1['availability'] = df1['availability'].fillna("Unknown")
    print("✅ Filled missing 'availability' in df1 with 'Unknown'")

# Fill missing first_publish_year in df2 with median
if df2['first_publish_year'].isnull().any():
    df2['first_publish_year'] = df2['first_publish_year'].fillna(df2['first_publish_year'].median())
    print("✅ Filled missing 'first_publish_year' in df2 with median")

# Standardize author names
df1['author'] = df1['author'].str.strip().str.title()
df2['author_email'] = df2['author_email'].str.lower().str.strip()
print("✅ Standardized author names and emails")

# Merge both DataFrames on 'book_id'
merged_df = pd.merge(df1, df2, on='book_id', how='inner')

# Drop duplicates if any
merged_df = merged_df.drop_duplicates()

# Show structure of final dataset
print("\n📊 Final merged dataset info:")
print(merged_df.info())

print("\n🔍 Sample of final cleaned and merged dataset:")
print(merged_df.head())


👉 Missing values in df1:
 book_id         0
title           1
author          0
price           1
availability    1
dtype: int64

👉 Missing values in df2:
 book_id               0
author_email          0
first_publish_year    1
genre                 0
dtype: int64

✅ Filled missing 'price' in df1 with mean
✅ Dropped rows with missing 'title' in df1
✅ Filled missing 'availability' in df1 with 'Unknown'
✅ Filled missing 'first_publish_year' in df2 with median
✅ Standardized author names and emails

📊 Final merged dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   book_id             4 non-null      int64  
 1   title               4 non-null      object 
 2   author              4 non-null      object 
 3   price               4 non-null      float64
 4   availability        4 non-null      object 
 5   author_email        4

In [21]:
from py2neo import Graph, Node, Relationship

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Clear existing data (optional if DB is already empty)
graph.delete_all()

# Create Authors
a1 = Node("Author", name="George Orwell")
a2 = Node("Author", name="J.K. Rowling")
a3 = Node("Author", name="Jane Austen")

# Create Books
b1 = Node("Book", title="1984", price=299, availability="In Stock", first_publish_year=1949)
b2 = Node("Book", title="Harry Potter", price=499, availability="Out of Stock", first_publish_year=1997)
b3 = Node("Book", title="Emma", price=350, availability="In Stock", first_publish_year=1815)

# Create Relationships
r1 = Relationship(a1, "WROTE", b1)
r2 = Relationship(a2, "WROTE", b2)
r3 = Relationship(a3, "WROTE", b3)

# Push to graph
graph.create(a1 | a2 | a3 | b1 | b2 | b3 | r1 | r2 | r3)


In [22]:
graph.run("MATCH (n) RETURN count(n)").to_data_frame()


,count(n)
0,6


In [14]:
query = """
MATCH (a:Author)-[:WROTE]->(b:Book)
RETURN a.name AS author, b.title AS title, b.price AS price, b.availability AS availability
LIMIT 10
"""
graph.run(query).to_data_frame()


,author,title,price,availability
0,George Orwell,1984,299,In Stock
1,Jane Austen,Emma,350,In Stock
2,J.K. Rowling,Harry Potter,499,Out of Stock
